In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
# Store csv created in part one into a DataFrame
file = pd.read_csv("city.csv")
file.dropna()
file.head()


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Hasaki,75,JP,1571277491,68,35.73,140.83,66.20,9.17
1,1,Ushuaia,30,AR,1571277491,57,-54.81,-68.31,48.20,9.17
2,2,Shu,98,KZ,1571277492,49,43.60,73.76,54.93,7.72
3,3,New Norfolk,40,AU,1571277492,62,-42.78,147.06,59.00,10.29
4,4,Kruisfontein,0,ZA,1571277492,34,-34.00,24.73,64.45,5.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Configure gmaps
gmaps.configure(api_key=g_key)


In [10]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [11]:
# Heatmap of humidity
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
temp_file=file.loc[(file['Max Temp']>=70)&(file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,rikitea,-23.12,-134.97,71.01,67,0,4.90,PF,1569425987
31,31,barrow,-38.31,-60.23,71.55,40,0,6.35,AR,1569426234
88,88,niteroi,-29.95,-51.16,79.00,53,0,3.36,BR,1569426245
160,160,shache,38.42,77.24,75.00,51,0,7.65,CN,1569426258
303,303,namibe,-15.19,12.15,70.47,88,0,9.48,AO,1569426288


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:

# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Hotel Name
1,rikitea,PF,-23.12,-134.97,
31,barrow,AR,-38.31,-60.23,
88,niteroi,BR,-29.95,-51.16,
160,shache,CN,38.42,77.24,
303,namibe,AO,-15.19,12.15,


In [9]:
# Set parameters to search for a hotel
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Iterate through 
# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df 


Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
1,rikitea,PF,-23.12,-134.97,Pension Maro'i
31,barrow,AR,-38.31,-60.23,
88,niteroi,BR,-29.95,-51.16,Hotel Express Canoas
160,shache,CN,38.42,77.24,Delong Hotel
303,namibe,AO,-15.19,12.15,Chik-Chik Namibe


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))